# 持久化选项
通过两种持久化方法：
* 快照
* 只追加文件 AOF append only file

## 快照持久化

### 创建快照的几种方法
* 客户端可以通过向redis发送BGSAVE命令来创建一个快照，此时会fork一个子进程来做存储的动作
* SAVE 不会用子进程，因此会使客户端不再相应命令，一般不常用，除非内存不够用
* 设置 save 配置选项: save 60 10000 指的是 当60秒内有10000次写入时，执行以下BGSAVE
* SHUTDOWN 会执行SAVE
* 当一个redis服务器连接另一个redis服务器，并向对方发送SYNC

### 几个适合快照的场景
* 个人开发
    * 例如设置 900 1 指的是900秒内如果有1条数据，则保存快照，如果没有，则在900秒后有一条数据进来就保存快照
* 对日志进行聚合计算
    * 首先确定日志丢失的可容忍程度，来决定存储快照的时间间隔
    * 需要恢复日志处理操作，知道存储快照时处理到了什么时候进度，此时需要知道被处理的文件和偏移量

In [1]:
import os
import redis
conn = redis.Redis()

In [2]:
def process_logs(conn, path, callback):
    # 获得当前的文档和offset
    current_file, offset = conn.mget('progress:file', 'progress:position')
    pipe = conn.pipeline()
    def update_progress():
        pipe.mset({
            'progress:file': fname,
            'progress:position': offset
        })
        pipe.execute()
    
    for fname in sorted(os.listdir(path)):
        if fname < current_file:
            continue
        with open(os.path.join(path, fname), 'rb') as inp:
            # 当是本文件，则从offset开始，否则从0开始
            if fname == current_file:
                inp.seek(int(offset, 10))
            else:
                offset = 0
            current_file = None
            for lno, line in enumerate(inp):
                callback(pipe, line)
                # offset指的是字符串的长度
                offset += int(offset) + len(line)
                if not (lno + 1) % 1000:
                    # 每1000行更新一下进度
                    update_grogress()
            update_progress()
            

### 大数据时
当数据量很大时，光创建子进程可能就要花去好多时间，如果内存不够用，可以选择关掉自动存快照，通过手动在不常用时间段 SAVE 的形式存储

## AOF持久化

* appendfsync: [always, everysec, no] 一般只推荐 everysec  
* AOF的问题就是，文件体积的大小，以及执行起来耗时间
    * 可以通过 BGREWRITEAOF 方法来去除冗余的命令
* 配置属性：auto-aof-rewrite-percentage / auto-aof-rewrite-min-size

# 复制

数据副本服务器处理客户端发送的读请求  
还需要检验硬盘写入

# 处理系统故障

* 验证快照和
* 主从切换/替换

# Redis 事务

在多个客户端同时处理相同的数据时，不谨慎的操作很容易导致数据出错。  
事务可以防止出错，并且还可以提升性能（延迟一次性提交，减少通信次数）。  
现在需要设计和实现一个商品买卖市场的方法  
## 数据结构
* 用户： Hash 包含用户的基本信息
* 用户包裹： Hash 包含所有商品
* 市场： Zset 成员：商品id + 所属者， 分值：价格

### 命令  
* WATCH： 表示监视某一个键，在MULTI前调用，如果在事务过程中该键有改动，服务器也不会马上返回改动信息，而是将该客户端的multi_dirty_cas 属性打开，代表处理的事务已经不安全了，接下来客户端在执行EXEC的时候，服务器先检查redis_dirty_cas，如果发现选项被打开，则返回WatchError，代表事务执行失败。    
* UNWATCH: WATCH会一直有效到下一个EXEC出现，但是有些时候事务执行不会出现EXEC（比如先做了一个条件判断发现条件不满足，就不执行了），此时为了不影响接下来的事务，需要调用UNWATCH来取消所有监视。
* DISCARD: 删除整个事务相关的内容，包括WATCH的设定

In [3]:
def list_items(conn, itemid, sellerid, price):
    """将商品放入市场的过程"""
    # 商人的包裹
    inventory = 'inventory:%s'%sellerid
    item = '%s.%s'%(itemid, sellerid)
    end = time.time() + 5
    pipe = conn.pipeline()
    while time.time() < end:
        try:
            # 开始监视商人的包裹，WATCH的出发，其实就意味着事务的开始
            # 在内部代码中 excute会对stack_commands进行打包，也就是在前后加上 MULTI 和 EXEC
            # 但是当WATCH方法被调用以后，在pipe.multi被调用以前的所有命令，并不会进入 stack_commands而是会直接执行
            pipe.watch(inventory)
            # 此时还没有进入事务模式，sismember还是事实返回的
            if not pipe.sismember(inventory, itemid):
                # 当 inventory 已经没有该物品了时，程序返回None
                pipe.unwatch()
                return None
            # 在调用了WATCH命令以后，必须有这么一个 multi方法，告诉程序接下来的命令需要被放进 stack_commonds里接受打包了
            # 如果是WatchError，说明在事务过程中这个键被动过，但是不代表这个键的这个商品被动过，因此需要循环
            pipe.multi()
            pipe.zadd('market:', item, price)
            pipe.srem(inventory, itemid)
            # 执行以后
            pipe.excute()
            return True
        except redis.Exception.WatchError:
            # 再次循环尝试，直到5秒过后
            pass
    # 5 秒过后还没有删除成功，返回False
    return False

## 购买商品

In [4]:
def purchase_item(conn, buyerid, itemid, sellerid, lprice):
    buyer = 'users:%s'%buyerid
    seller = 'users:%s'%sellerid
    item = '%s.%s'%(itemid, sellerid)
    inventory = 'inventory:%s'%buyerid
    end = time.time() + 10
    pipe = conn.pipeline()
    
    while time.time() < end:
        try:
            # 需要对两个：市场和买家，都进行监视
            pipe.watch('market:', buyer)
            
            price = pipe.zscore("market:", item)
            funds = int(pipe.hget(buyer, 'funds'))
            # 当两者的价格不相等，则终止交易
            if price != lprice or price > funds:
                pipe.unwatch()
                return None
            
            pipe.multi()
            pipe.hincrby(seller, 'funds', int(price))
            pipe.hincrby(buyer, 'funds', int(-price))
            pipe.sadd(inventory, itemid)
            pipe.zrem('market:', item)
            pipe.excute()
            return True
        except redis.exceptions.WatchError:
            pass
    return False

### 为什么redis没有实现加锁
* 加锁会阻塞别的客户端，很有可能造成长时间的等待
* 所以redis的watch方式可以减少大部分客户端的等待时间
* redis这种加锁方式交乐观锁，就属于失败了大不了重来
* 数据库的加锁方式叫做悲观锁

### pipe.multi 存在的意义
* 在内部代码中 excute会对stack_commands进行打包，也就是在前后加上 MULTI 和 EXEC
* 但是当WATCH方法被调用以后，在pipe.multi被调用以前的所有命令，并不会进入 stack_commands而是会直接执行
* 在调用了WATCH命令以后，必须有这么一个 multi 方法，告诉程序接下来的命令需要被放进 stack_commonds里接受打包了

# 非事务性流水线

虽然不是事务，但是流水线能够一次性将数据提交给服务器，减少两者之间的通信时间，所以也是能提高效率的    

In [5]:
# 使用非事务流水线的update_token
def update_token_pipeline(conn, token, user, item=None):
    timestamp = time.time()
    pipe = conn.pipeline(False)
    pipe.hset('login:', token, user)
    pipe.zadd('recent:', token, timestamp)
    if item:
        pipe.zadd('viewed:' + token, item, timestamp)
        pipe.zremrangebyrank('viewed' + token, 0, -26)
        pipe.zincrby('viewed:', item, -1)
    pipe.excute()

以上的优化会带来4-5倍的上升，取决于网络速度

# 性能方面的注意事项

可以通过 redis-benchmark 来直到自己的redis的性能特征，展现了在一秒内能够调用多少次一些常见的命令  
但是这里benchmark不会去处理返回过来的信息，所以整个性能测试是偏乐观的，同时也没有关网络方面的内容

* 性能的50%-60%: 没有使用流水线
* 性能的20%-30%/cannot assign requested address: 为每个命令创建了新的连接(python 内部自带池)